<a href="https://colab.research.google.com/github/ARSHITHbabu/CardioCortex/blob/main/CardioCortex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


# Load your dataset
data_path = '/content/heart_disease_uci.csv'
data = pd.read_csv(data_path)


# Data Preprocessing
data.fillna(data.mean(numeric_only=True), inplace=True)
for column in data.select_dtypes(include=['object']).columns:
    mode_value = data[column].mode()[0]
    data[column] = data[column].astype('category')
    data[column] = data[column].fillna(mode_value)

target_column = 'num'


if target_column not in data.columns:
    raise KeyError(f"The target column '{target_column}' was not found in the dataset.")


# Feature selection for prediction
X = data.drop(columns=[target_column, 'id', 'dataset'], errors='ignore')
y = data[target_column]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred)
rf_classification_report = classification_report(y_test, y_pred)

log_model = LogisticRegression(max_iter=5000)
log_model.fit(X_train_scaled, y_train)
log_pred = log_model.predict(X_test_scaled)

log_accuracy = accuracy_score(y_test, log_pred)
log_classification_report = classification_report(y_test, log_pred)


# Function to get user input
def get_user_input():
    print("\n" + "="*40)
    print("Please provide your health information:")
    print("="*40)

    user_age = int(input("\tEnter your age (0-120): "))
    user_sex = int(input("\tEnter your sex (0 for Female, 1 for Male): "))
    user_cp = int(input("\tEnter chest pain type (0: typical angina, 1: atypical angina, 2: non-anginal pain, 3: asymptomatic): "))
    user_trestbps = int(input("\tEnter resting blood pressure (mm Hg): "))
    user_chol = int(input("\tEnter cholesterol level (mg/dl): "))
    user_fbs = int(input("\tFasting blood sugar > 120 mg/dl (0: No, 1: Yes): "))
    user_restecg = int(input("\tResting electrocardiographic results (0: normal, 1: ST-T wave abnormality, 2: left ventricular hypertrophy): "))
    user_thalach = int(input("\tEnter maximum heart rate achieved (bpm): "))
    user_exang = int(input("\tExercise induced angina (0: No, 1: Yes): "))
    user_oldpeak = float(input("\tEnter oldpeak (depression induced by exercise): "))
    user_slope = int(input("\tSlope of peak exercise ST segment (0: upsloping, 1: flat, 2: downsloping): "))
    user_ca = int(input("\tEnter number of major vessels colored by fluoroscopy (0-3): "))
    user_thal = int(input("\tThalassemia (0: normal, 1: fixed defect, 2: reversible defect): "))

    print("\n" + "="*40)
    print("Thank you! You have entered the following information:")
    print("="*40)
    print(f"\tAge: {user_age}")
    print(f"\tSex: {'Female' if user_sex == 0 else 'Male'}")
    print(f"\tChest Pain Type: {user_cp}")
    print(f"\tResting Blood Pressure: {user_trestbps} mm Hg")
    print(f"\tCholesterol Level: {user_chol} mg/dl")
    print(f"\tFasting Blood Sugar > 120 mg/dl: {'Yes' if user_fbs == 1 else 'No'}")
    print(f"\tResting Electrocardiographic Results: {user_restecg}")
    print(f"\tMaximum Heart Rate Achieved: {user_thalach} bpm")
    print(f"\tExercise Induced Angina: {'Yes' if user_exang == 1 else 'No'}")
    print(f"\tOldpeak: {user_oldpeak}")
    print(f"\tSlope of Peak Exercise ST Segment: {user_slope}")
    print(f"\tNumber of Major Vessels Colored by Fluoroscopy: {user_ca}")
    print(f"\tThalassemia: {user_thal}")

    return {
        'age': user_age,
        'sex': user_sex,
        'cp': user_cp,
        'trestbps': user_trestbps,
        'chol': user_chol,
        'fbs': user_fbs,
        'restecg': user_restecg,
        'thalach': user_thalach,
        'exang': user_exang,
        'oldpeak': user_oldpeak,
        'slope': user_slope,
        'ca': user_ca,
        'thal': user_thal,
    }

user_data = get_user_input()

user_df = pd.DataFrame([user_data], columns=[
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'
])

user_df = pd.get_dummies(user_df, drop_first=True)

for col in X.columns:
    if col not in user_df.columns:
        user_df[col] = 0


user_df = user_df.reindex(columns=X.columns, fill_value=0)

user_df_scaled = scaler.transform(user_df)

user_prediction = rf_model.predict(user_df)
user_probability = rf_model.predict_proba(user_df)

predicted_class = user_prediction[0]
predicted_probabilities = user_probability[0]


health_condition = "No Heart Disease" if predicted_class == 0 else "Heart Disease Detected"

print("\n" + "="*40)
print("Model Evaluation Results")
print("="*40)

print(f"Random Forest Model Accuracy: {rf_accuracy:.2f}\n")
print("Random Forest Classification Report:")
print(rf_classification_report)

print("="*40)
print(f"Logistic Regression Model Accuracy: {log_accuracy:.2f}\n")
print("Logistic Regression Classification Report:")
print(log_classification_report)

print("="*40)
print("Prediction Results")
print("="*40)
print(f"Prediction: {health_condition}")
print(f"Class {predicted_class} Probability: {predicted_probabilities[predicted_class]:.2f}")
print(f"Class 1 Probability: {predicted_probabilities[1]:.2f}")
print(f"Class 2 Probability: {predicted_probabilities[2]:.2f}")
print(f"Class 3 Probability: {predicted_probabilities[3]:.2f}")
print(f"Class 4 Probability: {predicted_probabilities[4]:.2f}")
print("="*40)


# Function to provide health recommendations
def personalized_recommendations(age, cholesterol, sex, cp, trestbps, thalach, oldpeak, exang, ca, thal, predicted_class):
    recommendations = []

    if predicted_class == 0:
        recommendations.append("Great news! Your assessment indicates no heart disease. Here are some positive comments:")
        recommendations.append("1. Continue Healthy Habits: Keep up the good work! Maintaining a balanced diet and regular exercise is key.")
        recommendations.append("2. Stay Active: Engaging in regular physical activity will help you maintain your heart health.")
        recommendations.append("3. Regular Check-ups: Schedule routine health check-ups to stay on top of your health.")
        recommendations.append("4. Healthy Lifestyle: Continue avoiding smoking and limit alcohol intake to support your heart health.")
        recommendations.append("5. Monitor Stress Levels: Keep managing stress effectively, as it's vital for overall health.")

    elif predicted_class in [1, 2, 3, 4]:
        recommendations.append("Your assessment indicates potential heart disease. Here are some recommendations:")

        if age > 50:
            recommendations.append("• Regular health check-ups are recommended, especially for cardiovascular health.")

        if cholesterol > 240:
            recommendations.append("• High cholesterol level detected. Consider adopting a heart-healthy diet rich in fruits, vegetables, and whole grains.")
            recommendations.append("• Limit saturated fats and trans fats to help lower cholesterol levels.")
        elif cholesterol < 200:
            recommendations.append("• Great job! Your cholesterol level is within the normal range. Continue maintaining a balanced diet.")

        if sex == 1:
            recommendations.append("• As a male, you may be at a higher risk for heart disease. Regular screening is advised.")

        if cp in [1, 2]:
            recommendations.append("• Chest pain detected. Consult with a healthcare provider for further evaluation.")

        if trestbps > 140:
            recommendations.append("• High blood pressure detected. Lifestyle changes and medication may be necessary.")

        if oldpeak > 2:
            recommendations.append("• Elevated oldpeak suggests potential heart issues. Seek medical advice.")

        if exang == 1:
            recommendations.append("• Exercise-induced angina suggests further evaluation of your cardiovascular fitness is needed.")

        recommendations.append("• Maintain a balanced diet, rich in whole foods, low in processed foods.")
        recommendations.append("• Engage in regular physical activity, aiming for at least 150 minutes of moderate aerobic exercise weekly.")
        recommendations.append("• Avoid smoking and limit alcohol intake.")
        recommendations.append("• Manage stress through relaxation techniques, such as meditation or yoga.")

    return "\n".join(recommendations)

print("\n" + "="*40)
print("Prediction Results")
print("="*40)
print(f"Prediction: {health_condition}")
print(f"Class 0 Probability: {predicted_probabilities[0]:.2f}")
print(f"Class 1 Probability: {predicted_probabilities[1]:.2f}")

# Call the function to get recommendations
detailed_recommendations = personalized_recommendations(
    user_data['age'],
    user_data['chol'],
    user_data['sex'],
    user_data['cp'],
    user_data['trestbps'],
    user_data['thalach'],
    user_data['oldpeak'],
    user_data['exang'],
    user_data['ca'],
    user_data['thal'],
    predicted_class
)

print("\n" + "="*40)
print("Detailed Personalized Health Recommendations")
print("="*40)
print(detailed_recommendations)
print("="*40)





Please provide your health information:
	Enter your age (0-120): 69
	Enter your sex (0 for Female, 1 for Male): 1
	Enter chest pain type (0: typical angina, 1: atypical angina, 2: non-anginal pain, 3: asymptomatic): 2
	Enter resting blood pressure (mm Hg): 120
	Enter cholesterol level (mg/dl): 269
	Fasting blood sugar > 120 mg/dl (0: No, 1: Yes): 1
	Resting electrocardiographic results (0: normal, 1: ST-T wave abnormality, 2: left ventricular hypertrophy): 1
	Enter maximum heart rate achieved (bpm): 150
	Exercise induced angina (0: No, 1: Yes): 1
	Enter oldpeak (depression induced by exercise): 3.0
	Slope of peak exercise ST segment (0: upsloping, 1: flat, 2: downsloping): 2
	Enter number of major vessels colored by fluoroscopy (0-3): 2
	Thalassemia (0: normal, 1: fixed defect, 2: reversible defect): 2

Thank you! You have entered the following information:
	Age: 69
	Sex: Male
	Chest Pain Type: 2
	Resting Blood Pressure: 120 mm Hg
	Cholesterol Level: 269 mg/dl
	Fasting Blood Sugar > 1